#### Excercies 11 

Download Credit.csv file from ISL homepage. Create logistic regression models with possibly high prediction accuracy for predicting

1. if a given person has an income greater than 50 (hint: create new indicator variable),
2. how many credit cards a person has.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns
from sklearn import metrics
from patsy import dmatrices

%matplotlib inline

In [2]:
df = pd.read_csv('Credit.csv')
df['IncomeGreaterThan50'] = np.where(df['Income'] > 50.0, 'Yes', 'No')
df = df.astype({'IncomeGreaterThan50': 'category'})
df = pd.get_dummies(df, columns=['IncomeGreaterThan50'])

msk = np.random.rand(len(df)) < 0.8
train = df[msk]
test = df[~msk]


In [10]:
# a) Income greater than 50
equation = 'IncomeGreaterThan50_Yes ~ Rating + Balance + Limit'
y, X = dmatrices(equation, data=train, return_type='dataframe') 

# Use statsmodels logistic regression
logit = sm.Logit(y, X)
result = logit.fit()
# result.summary2()

inc_greater_true, X = dmatrices(equation, data=test, return_type='dataframe') 
inc_greater_pred = result.predict(X)


true = inc_greater_true['IncomeGreaterThan50_Yes'] == 1.0
pred = inc_greater_pred > 0.5

metrics.accuracy_score(true, pred)

Optimization terminated successfully.
         Current function value: 0.190301
         Iterations 9


0.9479166666666666

In [11]:
from sklearn import linear_model
from sklearn import metrics

# b) how many credit cards a person has
equation = 'Cards ~ Income + Limit + Student + Balance'

y, X = dmatrices(equation, data=train, return_type='dataframe') 


linreg = linear_model.LogisticRegression(
    C=1e9, 
    fit_intercept=False, 
    solver='liblinear', 
    max_iter=1000000, 
    multi_class='auto', 
    penalty='l1'
).fit(X, y.values.ravel())

y_true, X = dmatrices(equation, data=test, return_type='dataframe') 
y_pred = linreg.predict(X)

metrics.accuracy_score(y_true, y_pred)

0.3020833333333333

In [5]:
def get_logreg_acc(equation):
    y, X = dmatrices(equation, data=train, return_type='dataframe') 


    linreg = linear_model.LogisticRegression(
        C=1e9, 
        fit_intercept=False, 
        solver='liblinear', 
        max_iter=1000000, 
        multi_class='auto', 
        penalty='l1'
    ).fit(X, y.values.ravel())

    y_true, X = dmatrices(equation, data=test, return_type='dataframe') 
    y_pred = linreg.predict(X)

    return metrics.accuracy_score(y_true, y_pred)
    

In [12]:
import itertools
# 1
# 0.2597402597402597 Cards ~ Income
# 0.2857142857142857 Cards ~ Age
# 0.3246753246753247 Cards ~ Married
# 2
# 0.37662337662337664 Cards ~ Rating + Limit
# 3
# 0.38961038961038963 Cards ~ Rating + IncomeGreaterThan50_No + Limit
# 4
# 0.42857142857142855 Cards ~ Income + Limit + Student + Balance
# 5
# 6
# 7
# 8

predicted = 'Cards'
# operators = '+ - * / : **'.split()
features = 'Income Limit Rating Age Education Gender Student Married Ethnicity Balance IncomeGreaterThan50_No IncomeGreaterThan50_Yes'.split(' ')
best_result, best_equation = 0, ''

done = set()

for i in range(9, len(features)):
    print(i)
    for comb in itertools.combinations(features, i):
        comb = frozenset(comb)
        if comb in done:
            continue
        done.add(comb)
        with_operator = ' + '.join(comb)
        equation = (f'{predicted} ~ {with_operator}')
        result = get_logreg_acc(equation)
        if result > best_result:
            best_result = result
            best_equation = equation
            print(best_result, best_equation)
    done = set()

9
0.3541666666666667 Cards ~ Gender + Limit + Income + Education + Age + Rating + Married + Ethnicity + Student
0.3645833333333333 Cards ~ Gender + Limit + Income + Education + Age + Rating + Balance + Married + Student
0.375 Cards ~ Gender + Limit + IncomeGreaterThan50_No + Income + Education + Age + Rating + Balance + Student
0.3854166666666667 Cards ~ Gender + Limit + IncomeGreaterThan50_No + Income + Education + Age + Rating + Married + IncomeGreaterThan50_Yes
10
11
